In [6]:
from datasets import load_dataset

import os, zipfile, pathlib
from huggingface_hub import hf_hub_download
from datasets import Image

In [20]:
en = load_dataset("zhongshsh/CLoT-Oogiri-GO", data_files="en.jsonl", split="train")
print(en)

Dataset({
    features: ['text', 'question', 'star', 'type', 'image'],
    num_rows: 23769
})


In [37]:
t2t = en.filter(lambda x: x["type"] == "T2T")
i2t = en.filter(lambda x: x["type"] == "I2T")

In [9]:

zip_path = hf_hub_download(repo_id="zhongshsh/CLoT-Oogiri-GO", filename="images.zip", repo_type="dataset")

# 3b) extract once
root = pathlib.Path("./data/clot_images")
if not root.exists():
    with zipfile.ZipFile(zip_path) as zf:
        zf.extractall(root)

images.zip:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

In [55]:
exts = [".jpg", ".jpeg", ".png", ".webp"]
root = pathlib.Path("./data/clot_images")

def id_to_path(example):
    img_id = example["image"]
    if img_id is None:
        example["image_path"] = None
        return example
    
    p = root / f"{img_id}.jpg"

    #if p.exists():
    example["image_path"] = str(p)
    return example
    # fallback: keep None if not found
    example["image_path"] = "---"
    return example

i2t_local = i2t.map(id_to_path)
i2t_local = i2t_local.cast_column("image_path", Image())

In [58]:
i2t_local_df = i2t_local.to_pandas()
i2t_local_df.head()

,text,question,star,type,image,image_path
0,"It wasn't on purpose, I'm sorry!",None,5.0,I2T,5651380,"{'bytes': None, 'path': 'data/clot_images/5651..."
1,All matches on dating apps turn out to be rel...,None,NaN,I2T,007aPnLRgy1gs3ms5tjw6j30ci0elqa2,"{'bytes': None, 'path': 'data/clot_images/007a..."
2,"Today, the town is peaceful 💛 Peaceful 💛.",None,3.0,I2T,5856316,"{'bytes': None, 'path': 'data/clot_images/5856..."
3,This fell from above.,None,NaN,I2T,mntOVicn174EicqVzrSgro0GKsgy5X1Ynyeic2h1XicdmQ...,"{'bytes': None, 'path': 'data/clot_images/mntO..."
4,Give my stick back!,None,3.0,I2T,6531485,"{'bytes': None, 'path': 'data/clot_images/6531..."


In [36]:
print("T2T:", len(t2t), "I2T:", len(i2t_local))
print(i2t_local.select(range(3)))

T2T: 6433 I2T: 17336
Dataset({
    features: ['text', 'question', 'star', 'type', 'image', 'image_path'],
    num_rows: 3
})


In [28]:
# 3) convert to pandas
df_t2t = t2t.to_pandas()
df_i2t = i2t.to_pandas()  # keep only resolved paths

# 4) light cleanup and column order
# df_t2t = df_t2t[["type", "question", "text", "star"]]
# df_i2t = df_i2t[["type", "question", "image_path", "text", "star"]]

# # If you prefer comma-separated values:
# df_t2t.to_csv("clot_en_t2t.csv", index=False)
# df_i2t.to_csv("clot_en_i2t.csv", index=False)

In [29]:
df_t2t.head()

,text,question,star,type,image
0,"I bought something for 10,000 yen and received...",Just got annoyed for a little bit.,3,T2T,6029217
1,Payment only disappears at the time of payment.,I don't like mean girls.,5,T2T,7282923
2,Installation of dikes.,Don't do absolutely anything at a sushi restau...,3,T2T,6385428
3,The power is with the bureau chief.,I hate this boss from my part-time job.,3,T2T,3864126
4,I changed my name to Driving Delicious Techniq...,The reason why the reckless driver who caused ...,3,T2T,4349859


In [30]:
df_i2t.head()

,text,question,star,type,image
0,"It wasn't on purpose, I'm sorry!",None,5.0,I2T,5651380
1,All matches on dating apps turn out to be rel...,None,NaN,I2T,007aPnLRgy1gs3ms5tjw6j30ci0elqa2
2,"Today, the town is peaceful 💛 Peaceful 💛.",None,3.0,I2T,5856316
3,This fell from above.,None,NaN,I2T,mntOVicn174EicqVzrSgro0GKsgy5X1Ynyeic2h1XicdmQ...
4,Give my stick back!,None,3.0,I2T,6531485


In [34]:
i2t_local_df = i2t_local.to_pandas()
i2t_local_df

,text,question,star,type,image,image_path
0,"It wasn't on purpose, I'm sorry!",None,5.0,I2T,5651380,None
1,All matches on dating apps turn out to be rel...,None,NaN,I2T,007aPnLRgy1gs3ms5tjw6j30ci0elqa2,None
2,"Today, the town is peaceful 💛 Peaceful 💛.",None,3.0,I2T,5856316,None
3,This fell from above.,None,NaN,I2T,mntOVicn174EicqVzrSgro0GKsgy5X1Ynyeic2h1XicdmQ...,None
4,Give my stick back!,None,3.0,I2T,6531485,None
...,...,...,...,...,...,...
17331,Does it look like laundry?,None,4.0,I2T,6003560,None
17332,P.E. class followed by Chinese class.,None,NaN,I2T,007aPnLRgy1h1ro1td8acj30ci0elgod,None
17333,Typhoon! How about the name Typhoon?,None,3.0,I2T,3926868,None
17334,"Ever since Magneto moved upstairs, James with...",None,NaN,I2T,MjxjVcrFf9Ts8MeBl15Kibfia1tDETkH4RnHR4gZKukTbW...,None
